## ChirpNet Model 3 - CNN

By Thejaswin Kumaran<div>
Class: CS 4375.001 - Sriraam Natarajan

In [25]:
# Imports Cell
import os
import sys

import numpy as np
import pandas as pd
import librosa

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from sklearn import tree
# from sklearn.tree import DecisionTreeClassifier, export_text, DecisionTreeClassifier
# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
# from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier
# from decision_tree import load_monk_data



In [29]:

# Attributes Cell

# --- User Malleable parameters ---
# data_dir = 'jeantetDataset/Audio/Training'
data_dir = r'C:\Users\Thejas\Documents\Classes\4375_MachineLearning_Sriraam\Programs\ChirpNET\jeantetDataset\Audio\Training'
sr = 22050  # Sampling rate
duration = 5.0  # seconds (clips will be padded or trimmed to this length)
n_mels = 128  # Number of Mel bands
test_size = 0.2  # Fraction of data to reserve for validation
random_state = 42  # For reproducibility
# y_train = 0.8

samples_per_clip = int(sr * duration)
hop_length = 512  # Hop length for STFT

def preprocess(file_path):
    y, _ = librosa.load(file_path, sr=sr, duration=duration)

    # Standardize Audio Length
    if len(y) < samples_per_clip:
        y = np.pad(y, (0, samples_per_clip - len(y)), mode='constant')
    else:
        y = y[:samples_per_clip]

    # Construct Spectrograms
    mel_spec = librosa.feature.melspectrogram(
        y, sr=sr, n_mels=n_mels, hop_length=hop_length
    )

    mel_db = librosa.power_to_db(mel_spec, ref=np.max)
    return mel_db


In [ ]:
# 1) Gather file paths and labels
file_paths = []
labels = []
species = sorted(os.listdir(data_dir))  # Each subfolder is a species
for idx, sp in enumerate(species):
    sp_dir = os.path.join(data_dir, sp)
    if not os.path.isdir(sp_dir):
        continue
    for fname in os.listdir(sp_dir):
        if fname.lower().endswith('.mp3'):  # Change from '.wav' to '.mp3'
            file_paths.append(os.path.join(sp_dir, fname))
            labels.append(idx)


# Test Statements 1
print(f"Number of species: {len(species)}")
print(f"Species: {species}")
print(f"Number of files found: {len(file_paths)}")
print(f"Labels: {labels[:10]}")
print(f"/n/n")

# 2) Preprocess all files into spectrogram arrays
data = []
for fp in file_paths:
    spec = preprocess(fp)
    data.append(spec)

# Convert lists to numpy arrays
X = np.array(data)[..., np.newaxis]  # shape: (samples, n_mels, time_frames, 1)
y = to_categorical(labels, num_classes=len(species))

# Test Statements 2
print(f"X shape: {X.shape if len(data) > 0 else 'Empty'}")
print(f"y shape: {y.shape if len(labels) > 0 else 'Empty'}")

Number of species: 967
Species: ['Fringillidae_Serinus_canicollis_South Africa_2014-10-28_XC280667_song.mp3', 'Fringillidae_Serinus_canicollis_South Africa_2015-11-23_XC292869_song.mp3', 'Fringillidae_Serinus_canicollis_South Africa_2017-09-17_XC392698_song.mp3', 'Fringillidae_Serinus_canicollis_South Africa_2017-09-17_XC392699_song.mp3', 'Fringillidae_Serinus_canicollis_South Africa_2017-09-17_XC392700_song.mp3', 'Fringillidae_Serinus_canicollis_South Africa_2017-09-28_XC392701_song.mp3', 'Fringillidae_Serinus_canicollis_South Africa_2017-09-28_XC392702_song.mp3', 'Fringillidae_Serinus_canicollis_South Africa_2019-10-20_XC516046_song.mp3', 'Fringillidae_Serinus_canicollis_South Africa_2019-10-20_XC516047_song.mp3', 'Fringillidae_Serinus_canicollis_South Africa_2019-10-20_XC516048_call.mp3', 'Fringillidae_Serinus_canicollis_South Africa_2019-10-20_XC516049_call.mp3', 'Fringillidae_Serinus_canicollis_South Africa_2019-10-22_XC515435_adul.mp3', 'Fringillidae_Serinus_canicollis_South Afri

In [ ]:
# 3) Split into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=random_state, stratify=labels
)

# 4) Build the CNN model
input_shape = X_train.shape[1:]
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(species), activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# 5) Train the model
epochs = 10
batch_size = 32

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=epochs,
    batch_size=batch_size
)

# 6) Save the trained model
model.save('jeantet_cnn_model.h5')
print('Training complete. Model saved to jeantet_cnn_model.h5')


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.